## Run difference imaging on HiTS 2015 dataset

<br>Owner(s): **Weixiang Yu**
<br>Last Verified to Run: **2020-06-26**
<br>Verified Stack Release: **w_2020_24**

This notebook assumes that the raw images, coadd template, reference catalogs are in place. Creating template/coadd images will be another topic which is not shown here. This notebook follows instructions proivded in the __ap.pipe__ [documentation](https://pipelines.lsst.io/modules/lsst.ap.pipe/index.html), LSST Stack [tutorial](https://pipelines.lsst.io/getting-started/index.html), and take uses of the __obs_decam__ [repo](https://github.com/lsst/obs_decam/blob/master/README.md) 

## Set-up

You can find the Stack version that this notebook is running by using eups list -s on the terminal command line:

In [1]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list lsst_distrib -s

nb-wy73-w-2020-24
   19.0.0-1-gc154f85+75 	current w_2020_24 setup


We will need the following packages

In [2]:
import lsst.daf.persistence as dafPersist
import lsst.afw.image as afwImage

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import lsst.geom

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.time import Time

#%matplotlib widget
%matplotlib ipympl
#%matplotlib inline

In [3]:
import lsst.afw.display as afw_display
afw_display.setDefaultBackend('matplotlib')

## Create repo folder, injest/link data

In [4]:
repo_dir = '/project/stack-club/course_data/DIA/repo'

In [14]:
# create repo folder + specify the mapper
!mkdir /project/stack-club/course_data/DIA/repo
!echo "lsst.obs.decam.DecamMapper" > /project/stack-club/course_data/DIA/repo/_mapper

In [5]:
# explore avaiable raw data
!ls /project/mrawls/hits2015/ | grep "2015-"

2015-02-17
2015-02-18
2015-02-19
2015-02-20
2015-02-21
2015-02-22
2015-02-24
2015-02-27
2015-03-08
2015-03-09
2015-03-14


In [6]:
# ingest raw data
raw_args = repo_dir + " --filetype raw "+\
            "/project/mrawls/hits2015/2015*/g/*.fits.fz "+\
            "--mode=link"
!ingestImagesDecam.py {raw_args}

root INFO: Loading config overrride file '/opt/lsst/software/stack/stack/miniconda3-4.7.12-1a1d771/Linux64/obs_decam/19.0.0-33-g3d323b8/config/ingest.py'
CameraMapper INFO: Loading exposure registry from /project/stack-club/course_data/DIA/repo/registry.sqlite3
ingest.register INFO: Table "raw" exists.  Skipping creation
ingest INFO: /project/mrawls/hits2015/2015-02-17/g/decam0410915.fits.fz --<link>--> /project/stack-club/course_data/DIA/repo/2015-02-17/g/decam0410915.fits.fz
ingest INFO: /project/mrawls/hits2015/2015-02-17/g/decam0410929.fits.fz --<link>--> /project/stack-club/course_data/DIA/repo/2015-02-17/g/decam0410929.fits.fz
ingest INFO: /project/mrawls/hits2015/2015-02-17/g/decam0410931.fits.fz --<link>--> /project/stack-club/course_data/DIA/repo/2015-02-17/g/decam0410931.fits.fz
ingest INFO: /project/mrawls/hits2015/2015-02-17/g/decam0410971.fits.fz --<link>--> /project/stack-club/course_data/DIA/repo/2015-02-17/g/decam0410971.fits.fz
ingest INFO: /project/mrawls/hits2015/201

In [23]:
# injest calibration info -> create dir
calib_dir = os.path.join(repo_dir, 'calibs')
os.mkdir(calib_dir)

In [28]:
calib_args1 = repo_dir + " --calib " + calib_dir + " "\
        "/project/mrawls/hits2015/calib6/cpBIAS/*/BIAS*.fits " + \
        "--validity 999 --mode=link"

!ingestCalibs.py {calib_args1}

root INFO: Loading config overrride file '/opt/lsst/software/stack/stack/miniconda3-4.7.12-1a1d771/Linux64/obs_decam/19.0.0-33-g3d323b8/config/ingestCalibs.py'
CameraMapper INFO: Loading exposure registry from /project/stack-club/course_data/DIA/repo/registry.sqlite3
CameraMapper INFO: Loading calib registry from /project/stack-club/course_data/DIA/repo/calibs/calibRegistry.sqlite3
ingestCalibs.register INFO: Table "bias" exists.  Skipping creation
ingestCalibs.register INFO: Table "flat" exists.  Skipping creation
ingestCalibs.register INFO: Table "fringe" exists.  Skipping creation
ingestCalibs.register INFO: Table "dark" exists.  Skipping creation
ingestCalibs.register INFO: Table "illumcor" exists.  Skipping creation
ingestCalibs INFO: /project/mrawls/hits2015/calib6/cpBIAS/2015-02-16/BIAS-2015-02-16.fits --<link>--> /project/stack-club/course_data/DIA/repo/calibs/cpBIAS/2015-02-16/BIAS-2015-02-16.fits
ingestCalibs INFO: /project/mrawls/hits2015/calib6/cpBIAS/2015-02-17/BIAS-2015-0

In [29]:
calib_args2 = repo_dir + " --calib " + calib_dir + " " +\
        "/project/mrawls/hits2015/calib6/cpFLAT/*/*/FLAT*.fits " + \
        "--validity 999 --mode=link"

!ingestCalibs.py {calib_args2}

root INFO: Loading config overrride file '/opt/lsst/software/stack/stack/miniconda3-4.7.12-1a1d771/Linux64/obs_decam/19.0.0-33-g3d323b8/config/ingestCalibs.py'
CameraMapper INFO: Loading exposure registry from /project/stack-club/course_data/DIA/repo/registry.sqlite3
CameraMapper INFO: Loading calib registry from /project/stack-club/course_data/DIA/repo/calibs/calibRegistry.sqlite3
ingestCalibs.register INFO: Table "bias" exists.  Skipping creation
ingestCalibs.register INFO: Table "flat" exists.  Skipping creation
ingestCalibs.register INFO: Table "fringe" exists.  Skipping creation
ingestCalibs.register INFO: Table "dark" exists.  Skipping creation
ingestCalibs.register INFO: Table "illumcor" exists.  Skipping creation
ingestCalibs INFO: /project/mrawls/hits2015/calib6/cpFLAT/2015-02-16/g/FLAT-2015-02-16.fits --<link>--> /project/stack-club/course_data/DIA/repo/calibs/cpFLAT/2015-02-16/g/FLAT-2015-02-16.fits
ingestCalibs INFO: /project/mrawls/hits2015/calib6/cpFLAT/2015-02-16/i/FLAT-

In [38]:
calib_args3 = repo_dir + " --calib " + calib_dir + " " +\
        "--calibType defect " +\
        "/project/mrawls/hits2015/calib6/defects/*/*.fits " + \
        "--validity 0 --mode=skip"

!ingestCalibs.py {calib_args3}

root INFO: Loading config overrride file '/opt/lsst/software/stack/stack/miniconda3-4.7.12-1a1d771/Linux64/obs_decam/19.0.0-33-g3d323b8/config/ingestCalibs.py'
usage: ingestCalibs.py input [options]
ingestCalibs.py: error: unrecognized arguments: --calibType
